In [1]:
sc.textFile('data/coupon150720.csv').first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [8]:
from time import strptime
from time import mktime

In [5]:
d1 = strptime('150904','%y%m%d')

In [6]:
d2 = strptime('150623','%y%m%d')

In [11]:
(mktime(d1)-mktime(d2))/(3600*24)

73.0

In [12]:
sc.textFile('data/transm150720.csv').first()

u'79062005698500,TKTT,30,150719,FR,0.0,EUR,T,T,141025,PARA127A8,0.0,EUR,   ,EX,,150719,0.0'

In [13]:
def getTCNSalida(l):
    elems = l.split(",")
    tcn = elems[0]
    salida = elems[12]
    return (tcn, salida)

In [14]:
cs = sc.textFile('data/coupon150720.csv').map(getTCNSalida)

In [15]:
cs.take(2)

[(u'79062005698500', u'150904'), (u'79062005698500', u'150905')]

In [16]:
def getTCNCompra(l):
    elems = l.split(",")
    tcn = elems[0]
    compra = elems[9]
    return (tcn, compra)

In [17]:
ts = sc.textFile('data/transm150720.csv').map(getTCNCompra)

In [18]:
ts.take(2)

[(u'79062005698500', u'141025'), (u'79062005924069', u'150719')]

In [19]:
cs.take(2)

[(u'79062005698500', u'150904'), (u'79062005698500', u'150905')]

In [22]:
def fechaMenor(ds):    
    segs = [mktime(strptime(d,'%y%m%d')) for d in ds]    
    return min(segs)

In [24]:
def diasDeDiferencia(fechaCompraStr, fechaSalidaSegs):
    fechaCompraStruct = strptime(fechaCompraStr, '%y%m%d')
    fechaCompraSegs = mkTime(fechaCompraStruct)
    return (fechaSalidaSegs-fechaCompraSegs)/(3600*24)

In [34]:
fts = ts.filter(lambda x: x[1] != '')
fcs = cs.filter(lambda x: x[1] != '')

In [35]:
r = fts.cogroup(fcs).mapValues(lambda x: fechaMenor(x[1])-fechaMenor(x[0]))

In [39]:
r.take(5)

[(u'79062005994272', 86400.0),
 (u'79065668614400', 432000.0),
 (u'79062005879562', 3888000.0),
 (u'79062005474691', 3974400.0),
 (u'79062006175895', 2851200.0)]

In [43]:
h = r.mapValues(lambda dif: dif/(3600*24)).map(lambda x: (int(x[1]),1)) \
.reduceByKey(lambda x,y: x+y)

In [42]:
r.mapValues(lambda dif: dif/(3600*24)).map(lambda x: (int(x[1]),1)).take(5)

[(1, 1), (5, 1), (45, 1), (46, 1), (33, 1)]

In [44]:
h.take(5)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 27.0 failed 1 times, most recent failure: Lost task 0.0 in stage 27.0 (TID 41, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/pyspark/rdd.py", line 2355, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/spark/python/pyspark/rdd.py", line 2355, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/spark/python/pyspark/rdd.py", line 317, in func
    return f(iterator)
  File "/usr/local/spark/python/pyspark/rdd.py", 